In [8]:
import uuid
import json
import shapely
import geopandas as gpd
from odp.client import OdpClient
from odp.client.dto.table_spec import TableSpec
from odp.dto.catalog import DataCollectionDto, DatasetDto

## Create resource in ODP Catalogue

In [9]:
client = OdpClient()
uuid_str = str(uuid.uuid4())


# create collection
with open("collection.json") as file:
    collection_config = json.load(file)

# randomize collection name:
collection_config["metadata"]["name"] = collection_config["metadata"]["name"] + "-" + uuid_str

collection_manifest = DataCollectionDto(**collection_config)
collection_dto = client.catalog.create(collection_manifest)


# create dataset
with open("dataset.json") as file:
    dataset_config = json.load(file)
    table_schema = dataset_config["table_schema"]
    dataset_config = dataset_config["dataset_config"]

# randomize dataset name:
dataset_config["metadata"]["name"] = dataset_config["metadata"]["name"] + "-" + uuid_str
# link dataset to collection
dataset_config["spec"]["data_collection"] = dataset_config["spec"]["data_collection"] + collection_config["metadata"]["name"]

dataset_manifest = DatasetDto(**dataset_config)
dataset_dto = client.catalog.create(dataset_manifest)
table_spec = TableSpec(table_schema=table_schema)
client.tabular.create_schema(resource_dto=dataset_dto, table_spec=table_spec)

TableSpec(table_schema={'record': {'type': 'long'}, 'month': {'type': 'long'}, 'height': {'type': 'double'}, 'ref': {'type': 'double'}, 'ax': {'type': 'double'}, 'gx': {'type': 'double'}, 'mz': {'type': 'double'}, 'gy': {'type': 'double'}, 'year': {'type': 'long'}, 'rssi': {'type': 'long'}, 'temp_imu': {'type': 'double'}, 'validity': {'type': 'double'}, 'temp_2': {'type': 'double'}, 'temp_4': {'type': 'double'}, 'atm': {'type': 'double'}, 'day': {'type': 'long'}, 'satellites': {'type': 'long'}, 'temp_1': {'type': 'double'}, 'speed': {'type': 'double'}, 'temp_3': {'type': 'double'}, 'pdop': {'type': 'double'}, 'ay': {'type': 'double'}, 'mx': {'type': 'double'}, 'course': {'type': 'double'}, 'minute': {'type': 'long'}, 'hour': {'type': 'long'}, 'gz': {'type': 'double'}, 'geometry': {'type': 'geometry'}, 'hdop': {'type': 'double'}, 'az': {'type': 'double'}, 'vdop': {'type': 'double'}, 'second': {'type': 'long'}, 'sep': {'type': 'double'}, 'my': {'type': 'double'}, 'timestamp': {'type': 't

## Prepare dataframe and load to ODP

In [12]:
def safe_to_geojson(geometry):
    if not geometry:
        return None
    try:
        return json.loads(shapely.to_geojson(geometry))
    except Exception:
        return None

gdf = gpd.read_parquet("iliad.parquet")
gdf["geometry"] = gdf["geometry"].apply(safe_to_geojson)
datapoints_to_write = gdf.to_dict("records")
client.tabular.write(dataset_dto, datapoints_to_write)


/tmp/ipykernel_523/611474191.py:10: UserWarning: Geometry column does not contain geometry.
  gdf["geometry"] = gdf["geometry"].apply(safe_to_geojson)
